In [1]:
import pickle
import numpy as np
import torch

In [2]:
with open('./combined_feats_train_knn.pkl', 'rb') as fp:
    combined_feats_train_knn = pickle.load(fp)

with open('./labels_train_knn.pkl', 'rb') as fp:
    labels_train_knn = pickle.load(fp)

with open('./model_outputs_train_knn.pkl', 'rb') as fp:
    model_outputs_train_knn = pickle.load(fp)

with open('./combined_feats_eval.pkl', 'rb') as fp:
    combined_feats_eval = pickle.load(fp)

with open('./labels_eval.pkl', 'rb') as fp:
    labels_eval = pickle.load(fp)

with open('./model_outputs_eval.pkl', 'rb') as fp:
    model_outputs_eval = pickle.load(fp)

/Users/sriram/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from sklearn.neighbors import KNeighborsClassifier
k = 3
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(combined_feats_train_knn, labels_train_knn)
neighbors_dist, neighbors_ind = knn_classifier.kneighbors(X=combined_feats_eval)

In [6]:
hits = 0
for i, neighbors_ind_i in enumerate(neighbors_ind):
    ind_model_accs = (model_outputs_train_knn[neighbors_ind_i] == labels_train_knn[neighbors_ind_i].unsqueeze(1)).float().mean(dim=0)
    ind_model_accs_maxinds = torch.argsort(ind_model_accs, descending=True)
    hits += (model_outputs_eval[i][ind_model_accs_maxinds].mean(dim=0).argmax() == labels_eval[i]).mean()

In [ ]:
print(hits/len(neighbors_ind))